# 数据处理

In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from utils import *
from imblearn import over_sampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MultiLabelBinarizer,LabelBinarizer
import emoji

## 数据清洗1：把所有的表情换成xx

In [2]:
df=pd.read_csv('data/ctrip/data1119.csv')
df['reviews'] = df['reviews'].map(lambda x: filter_emoji(x,restr='xx'))
write_txt(df,path='data/ctrip/total1119.txt')

## 数据清洗2：保留所有表情转而更正所有index

In [33]:
df=pd.read_csv('data/ctrip/data1119.csv')
df['aspect_opinion_list']=df.apply(lambda x:correct_idx(x.reviews,x.aspect_opinion_list),axis=1)
df.to_csv('data/ctrip/1119clean.csv',index=False)
false=clean('data/ctrip/1119clean.csv')
write_txt2(df,false)

## 训练集和测试集划分

In [3]:
write_train_test(train_path='data/ctrip/train.txt',test_path='data/ctrip/test.txt',root='data/ctrip/total1119.txt')

In [4]:
a=read_txt('data/ctrip/train.txt')
print(len(a))
a=read_txt('data/ctrip/test.txt')
print(len(a))
a=read_txt('data/ctrip/total1119.txt')
print(len(a))

17757
4440
22197


In [6]:
!mkdir -p ./data/tasd-cn/ctrip
!cp data/ctrip/train.txt data/ctrip/test.txt ./data/tasd-cn/ctrip/
!cp data/ctrip/test.txt ./data/tasd-cn/ctrip/dev.txt

## 抽取式数据处理过程

In [3]:
sents, labels=['Screen is awesome , battery life is good .'],[[([0], [2], 'POS'), ([4, 5], [7], 'POS')]]
sents[0]=sents[0].split(' ')
a=get_annotated_aste_targets(sents, labels)
print(a)

[['[Screen|positive|awesome]', 'is', 'awesome', ',', '[battery', 'life|positive|good]', 'is', 'good', '.']]


## 数据均衡（重采样）

In [37]:
sents,lab,total=read_line_examples_from_file('./data/tasd-cn/ctrip/train.txt')

Total examples = 17757


In [44]:
dic,num=cal_rate(lab)
keys=list(dic.keys())
# x,y=build_xy(total,lab)
x,labs=build_xy(total,lab)
x=np.array(x)
x=x.reshape((-1,1))
print(x.shape)
y=[]
for i in labs:
    y.append(keys.index(i))

(20020, 1)


In [46]:
ros = over_sampling.RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(x, y)
# sorted(Counter(y_resampled).items())
X_resampled.shape

(613190, 1)

In [56]:
write_txt(X_resampled)

## endpoint调用

In [3]:

from boto3.session import Session
import json
# df=pd.read_csv('./data/hp/summary/news_summary_cleaned_small_test.csv')
# print('原文:',df.loc[0,'text'])
# print('真实标签:',df.loc[0,'headlines'])
data={"data": '早餐一般般，勉勉强强填饱肚子，样式可选性不多，可能是疫情的影响吧。不过酒店的服务不错，五个小孩早餐都送了，点👍。由于酒店历史有点长，所以设施感觉一般般，整体还可以，三钻吧'}
session = Session()
    
runtime = session.client("runtime.sagemaker")
response = runtime.invoke_endpoint(
    EndpointName='absa',
    ContentType="application/json",
    Body=json.dumps(data),
)

result = json.loads(response["Body"].read())
print (result)

{'result': '(小孩早餐, 儿童餐饮, 送了点👍, 正)', 'infer_time': '0:00:00.725859'}
